Inputs:
- Shapefile

STAC search parameters:
- Time period
- Satellite sensor
- Target band indexes

Outputs:
- CSV containing a multiindex of zonal statistics for the specified polygons over the specified time period

In [32]:
import rioxarray
import xarray as xr
from dask.distributed import Client as DaskClient, LocalCluster
import warnings
warnings.filterwarnings('ignore')
xr.set_options(keep_attrs=True)
cluster = LocalCluster(n_workers=4, threads_per_worker=4)
client = DaskClient(cluster); client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33365/status,
Dashboard: http://127.0.0.1:33365/status,Workers: 4
Total threads: 16,Total memory: 15.34 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39033,Workers: 0
Dashboard: http://127.0.0.1:33365/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:43651,Total threads: 4
Dashboard: http://127.0.0.1:33937/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:42235,


In [22]:
# Read shapefile as a GeoDataFrame
import geopandas as gpd
from pathlib import Path

# Path to the shapefile (relative to the repository root)
shp_path = Path("inputs/forest283Blackbutt/forest283.shp")

# Sanity check the path exists to give a clear error if not
if not shp_path.exists():
    raise FileNotFoundError(f"Shapefile not found: {shp_path.resolve()}")

# Read the shapefile
gdf = gpd.read_file(shp_path)

# Quick summary
print(f"Loaded {len(gdf)} features | CRS: {gdf.crs}")

gdf.head()

Loaded 858 features | CRS: EPSG:7844


,UNIT_KEY,PRUK,FOREST,LOG_AREA,CPT_ID,CPT_NO,SUB,SLASH,CPTCOMM,PRA,...,Shape__Are,Shape__Len,PRIM_SPEC_,SEC_SPEC_C,TCPT,SHORT_DESC,LOCATION,ST_ASSETID,T1GIS_ID,geometry
0,283_BIDWILLII_4_1,283_BIDWILLII_4,283,BIDWILLII,4,4,None,None,None,1,...,0.000010,0.020720,96,0,004,BIDWILLII_4_1,SF283\BIDWILLII\004,ST005076,5671,"POLYGON ((152.15976 -26.77157, 152.15978 -26.7..."
1,283_BIDWILLII_4_3,283_BIDWILLII_4,283,BIDWILLII,4,4,None,None,None,3,...,0.000003,0.008107,96,0,004,BIDWILLII_4_3,SF283\BIDWILLII\004,ST005078,6680,"POLYGON ((152.1705 -26.77303, 152.17048 -26.77..."
2,283_BIDWILLII_4_1,283_BIDWILLII_4,283,BIDWILLII,4,4,None,None,None,1,...,0.000025,0.042623,96,0,004,BIDWILLII_4_1,SF283\BIDWILLII\004,ST005076,5671,"POLYGON ((152.16206 -26.77525, 152.16215 -26.7..."
3,283_BIDWILLII_4_2,283_BIDWILLII_4,283,BIDWILLII,4,4,None,None,None,2,...,0.000004,0.007995,96,0,004,BIDWILLII_4_2,SF283\BIDWILLII\004,ST005077,10668,"POLYGON ((152.17142 -26.77423, 152.17141 -26.7..."
4,283_BIDWILLII_4_3,283_BIDWILLII_4,283,BIDWILLII,4,4,None,None,None,3,...,0.000004,0.008075,96,0,004,BIDWILLII_4_3,SF283\BIDWILLII\004,ST005078,6680,"POLYGON ((152.16735 -26.77325, 152.16752 -26.7..."


In [23]:
bbox = gdf.dissolve().total_bounds.tolist()

## Load in the STAC connection

In [5]:
from pystac_client import Client as StacClient
import planetary_computer

In [25]:
# catalog = StacClient.open('https://planetarycomputer.microsoft.com/api/stac/v1/', modifier=planetary_computer.sign_inplace)
# catalog

catalog = StacClient.open("https://explorer.dea.ga.gov.au/stac")
import odc.stac
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

catalog

<Client id=AWS_Explorer>

## Search the catalog for what we want

In [ ]:
# Sentinel 2B

stac_load(items=list(catalog.search(
    bbox=bbox,
    collections=["ga_s2bm_ard_3"],
    datetime="2025-11-01/2025-11-30",
).get_items()), bands=['nbart_blue', 'nbart_red', 'nbart_nir_1', 'nbart_swir_2', 'nbart_swir_3', 'nbart_green'], bbox=bbox, groupby='time', chunks={'time': 1, 'x': 4096, 'y': 4096})

# Data appears to start 2017-07-09 and as of 2025-11-19 there was data available for 2025-11-14

<xarray.Dataset> Size: 100MB
Dimensions:       (y: 2410, x: 864, time: 2)
Coordinates:
  * y             (y) float64 19kB 7.039e+06 7.039e+06 ... 7.015e+06 7.015e+06
  * x             (x) float64 7kB 4.137e+05 4.137e+05 ... 4.223e+05 4.223e+05
  * time          (time) datetime64[ns] 16B 2025-11-04T00:04:05.148537 2025-1...
    spatial_ref   int32 4B 32756
Data variables:
    nbart_blue    (time, y, x) float32 17MB dask.array<chunksize=(1, 2410, 864), meta=np.ndarray>
    nbart_red     (time, y, x) float32 17MB dask.array<chunksize=(1, 2410, 864), meta=np.ndarray>
    nbart_nir_1   (time, y, x) float32 17MB dask.array<chunksize=(1, 2410, 864), meta=np.ndarray>
    nbart_swir_2  (time, y, x) float32 17MB dask.array<chunksize=(1, 2410, 864), meta=np.ndarray>
    nbart_swir_3  (time, y, x) float32 17MB dask.array<chunksize=(1, 2410, 864), meta=np.ndarray>
    nbart_green   (time, y, x) float32 17MB dask.array<chunksize=(1, 2410, 864), meta=np.ndarray>

In [33]:
# results = catalog.search(
#     collections=['landsat-c2-l2'],
#     bbox=bbox,
#     datetime=['2015-01-01', '2025-06-30'],
#     query={'eo:cloud_cover': {'lt': 20}, 'platform': {'in': ['landsat-8']}}
# )

results = catalog.search(
    bbox=bbox,
    collections=["ga_ls8c_ard_3"],
    datetime="2025-01-01/2025-11-01",
)

items = list(results.get_items())
len(items), items[0] if items else None

(57, <Item id=97d8f84f-4600-400e-9461-c1fea96b9104>)

## Load data cube

In [34]:
from odc.stac import stac_load

# Enhanced bands list for comprehensive plant health monitoring
# bands = ['blue', 'red', 'nir08', 'swir16', 'swir22', 'green']  # Added SWIR and green
# chunks = {'time': 1, 'x': 2048, 'y': 2048}

bands = ['nbart_blue', 'nbart_red', 'nbart_nir', 'nbart_swir_1', 'nbart_swir_2', 'nbart_green']
chunks = {'time': 1, 'x': 4096, 'y': 4096}

data = stac_load(items=items, bands=bands, bbox=bbox, groupby='time', chunks=chunks)
data

<xarray.Dataset> Size: 317MB
Dimensions:       (y: 805, x: 288, time: 57)
Coordinates:
  * y             (y) float64 6kB -2.961e+06 -2.961e+06 ... -2.985e+06
  * x             (x) float64 2kB 4.137e+05 4.137e+05 ... 4.222e+05 4.223e+05
  * time          (time) datetime64[ns] 456B 2025-01-06T23:42:11.885520 ... 2...
    spatial_ref   int32 4B 32656
Data variables:
    nbart_blue    (time, y, x) float32 53MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_red     (time, y, x) float32 53MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_nir     (time, y, x) float32 53MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_swir_1  (time, y, x) float32 53MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_swir_2  (time, y, x) float32 53MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_green   (time, y, x) float32 53MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>

In [35]:
print(f"Size of data: {data.nbytes / (1024 ** 2):.2f} MB")

Size of data: 302.47 MB


## Aggregate to monthly

In [36]:
data_monthly = data.resample(time='1M').median()
data_monthly

<xarray.Dataset> Size: 56MB
Dimensions:       (time: 10, y: 805, x: 288)
Coordinates:
  * time          (time) datetime64[ns] 80B 2025-01-31 2025-02-28 ... 2025-10-31
  * y             (y) float64 6kB -2.961e+06 -2.961e+06 ... -2.985e+06
  * x             (x) float64 2kB 4.137e+05 4.137e+05 ... 4.222e+05 4.223e+05
    spatial_ref   int32 4B 32656
Data variables:
    nbart_blue    (time, y, x) float32 9MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_red     (time, y, x) float32 9MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_nir     (time, y, x) float32 9MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_swir_1  (time, y, x) float32 9MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_swir_2  (time, y, x) float32 9MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>
    nbart_green   (time, y, x) float32 9MB dask.array<chunksize=(1, 805, 288), meta=np.ndarray>

## Fetch the data

In [30]:
from importlib import reload
import zonalStatistics
reload(zonalStatistics)

<module 'zonalStatistics' from '/mnt/c/Users/rossb/OneDrive - HQPLANTATIONS PTY LTD/Documents/code/my_code/satellite-data-toolbox/zonalStatistics.py'>

In [37]:
# Efficient zonal statistics for raw bands (not indices)
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

aoi_crs = gdf.crs or 'EPSG:4326'

# First, let's update the data loading to include more bands
print("Current bands in data:", list(data_monthly.data_vars))

# Use the function to extract band values
print("Computing zonal statistics (streaming to per-unit CSVs)...")
zonalStatistics.compute_zonal_stats_bands(
    data_monthly=data_monthly,
    gdf=gdf[:5],
    bands=bands,
    aoi_crs=gdf.crs,
    output_dir="outputs",
    verbose=True,
    logger=None
)

Current bands in data: ['nbart_blue', 'nbart_red', 'nbart_nir', 'nbart_swir_1', 'nbart_swir_2', 'nbart_green']
Computing zonal statistics (streaming to per-unit CSVs)...


  0%|          | 0/5 [00:00<?, ?it/s]/home/ben/miniconda3/envs/forestHealth/lib/python3.14/site-packages/dask/_task_spec.py:758: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)
/home/ben/miniconda3/envs/forestHealth/lib/python3.14/site-packages/dask/_task_spec.py:758: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)
/home/ben/miniconda3/envs/forestHealth/lib/python3.14/site-packages/dask/_task_spec.py:758: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)
/home/ben/miniconda3/envs/forestHealth/lib/python3.14/site-packages/dask/_task_spec.py:758: RuntimeWarning: All-NaN slice encountered
  return self.func(*new_argspec, **kwargs)
100%|██████████| 5/5 [02:16<00:00, 27.40s/it]


In [ ]:
# data_monthly[bands].rio.clip([shp_mapping(gdf.iloc[0].geometry)], aoi_crs, drop=True).mean(dim=['y', 'x'])

In [38]:
# Clean the dataframe by removing invalid values
df_bands_cleaned = df_bands.dropna()  # Remove nulls

# Define realistic value ranges for each band (Landsat-8 surface reflectance values)
band_thresholds = {
    'blue': (0, 10000),
    'green': (0, 10000), 
    'red': (0, 10000),
    'nir08': (0, 10000),
    'swir16': (0, 10000),
    'swir22': (0, 10000)
}

# Apply thresholds to both mean and median columns
initial_count = len(df_bands_cleaned)
for band, (min_val, max_val) in band_thresholds.items():
    for stat in ['mean', 'median']:
        col_name = f'{band}_{stat}'
        if col_name in df_bands_cleaned.columns:
            df_bands_cleaned = df_bands_cleaned[
                df_bands_cleaned[col_name].between(min_val, max_val)
            ]

print(f"Original records: {len(df_bands)}")
print(f"After cleaning: {len(df_bands_cleaned)}")
print(f"Removed {len(df_bands) - len(df_bands_cleaned)} invalid records")

# Show column structure
print(f"\nColumns in cleaned dataset: {list(df_bands_cleaned.columns)}")
df_bands_cleaned.head()

NameError: name 'df_bands' is not defined

In [ ]:
df = df_bands_cleaned.copy()

In [ ]:
print(f"Processed {len(gdf)} units with {len(df)} total records")
print(f"Unique units: {df['unit_key'].unique()}")
df.head()

# Export individual CSV files for each UNIT_KEY
from pathlib import Path

# Create outputs directory if it doesn't exist
outputs_dir = Path("outputs")
outputs_dir.mkdir(exist_ok=True)

# Get all band columns (mean and median)
band_columns = [col for col in df.columns 
                if any(band in col for band in ['blue', 'green', 'red', 'nir08', 'swir16', 'swir22'])
                and any(stat in col for stat in ['mean', 'median'])]

print(f"Band columns to export: {band_columns}")

# Export CSV for each unique UNIT_KEY
for unit_key in df['unit_key'].unique():
    if pd.notna(unit_key):  # Skip any NaN values
        # Filter data for this unit
        unit_data = df[df['unit_key'] == unit_key].copy()
        unit_data = unit_data.sort_values('time')
        
        # Select relevant columns (time + all band statistics)
        export_cols = ['time'] + band_columns
        unit_export = unit_data[export_cols]
        
        # Create filename (sanitize unit_key for filesystem)
        safe_unit_key = str(unit_key).replace('/', '_').replace('\\', '_').replace(' ', '_')
        filename = f"BANDS_{safe_unit_key}.csv"
        filepath = outputs_dir / filename
        
        # Export to CSV
        unit_export.to_csv(filepath, index=False)
        print(f"Exported: {filepath}")

print(f"\nExported {len(df['unit_key'].unique())} CSV files to {outputs_dir.resolve()}")

# Also export combined CSV with all units and bands
combined_filepath = outputs_dir / "BANDS_all_units.csv"
export_cols = ['time', 'unit_key'] + band_columns
df[export_cols].to_csv(combined_filepath, index=False)
print(f"Exported combined band data: {combined_filepath}")

# Show sample of what we're exporting
print(f"\nSample of exported data structure:")
print(df[['time', 'unit_key'] + band_columns[:4]].head())

Processed 858 units with 0 total records


KeyError: 'unit_key'